In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import datasets,layers,Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,GlobalAveragePooling2D
from zipfile import ZipFile
import os,glob
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.models import Model
from sklearn import preprocessing
from keras.layers import BatchNormalization
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.utils import to_categorical

<ipython-input-1-c60e90ce3514>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook as tqdm


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 76 bytes


In [ ]:
!kaggle competitions download -c ai-mlprocom24

100% 379M/380M [00:18<00:00, 24.1MB/s]
100% 380M/380M [00:18<00:00, 22.1MB/s]


In [ ]:
file = "/content/ai-mlprocom24.zip"
with ZipFile(file,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

# Load the CSV file containing image names and labels for training
train_df = pd.read_csv('Train.csv')

# Load the images and extract features using a pre-trained ResNet50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=resnet.input, outputs=resnet.output)

features_resnet = []
labels = []

for index, row in train_df.iterrows():
    image_path = os.path.join('/content/train_images', row['ID'])
    image = cv2.imread(image_path)
    if image is not None:
        image = cv2.resize(image, (224, 224))  # Resize the image to match ResNet50 input size
        image = np.expand_dims(image, axis=0)  # Add batch dimension
        features = model.predict(image)
        features_resnet.append(features.flatten())
        labels.append(row['Label'])

features_resnet = np.array(features_resnet)
labels = np.array(labels)

# Convert labels to numeric format
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features_resnet, labels, test_size=0.2, random_state=42)

# Train a LightGBM model using the extracted features
model_lgbm = LGBMClassifier()
model_lgbm.fit(X_train, y_train)

# Evaluate the model
y_pred = model_lgbm.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

# Use the trained model to make predictions on the test images
test_df = pd.read_csv('Test.csv')
features_test = []

for index, row in test_df.iterrows():
    image_path = os.path.join('/content/test_images', row['ID'])
    image = cv2.imread(image_path)
    if image is not None:
        image = cv2.resize(image, (224, 224))  # Resize the image to match ResNet50 input size
        image = np.expand_dims(image, axis=0)  # Add batch dimension
        features = model.predict(image)
        features_test.append(features.flatten())

features_test = np.array(features_test)

# Make predictions
predictions = model_lgbm.predict(features_test)

# Convert numeric predictions back to original labels
predicted_labels = label_encoder.inverse_transform(predictions)

# Save the predictions to a CSV file
test_df['Label'] = predicted_labels
test_df.to_csv('Test_Predictions_LGM.csv', index=False)


1/1 [==============================] - 0s 22ms/step
[LightGBM] [Info] Number of positive: 31, number of negative: 34
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021080 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 207745
[LightGBM] [Info] Number of data points in the train set: 65, number of used features: 17225
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.476923 -> initscore=-0.092373
[LightGBM] [Info] Start training from score -0.092373
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning

# **SVC + Resnet50**

In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

# Load the CSV file containing image names and labels for training
train_df = pd.read_csv('Train.csv')

# Load the images and extract features using a pre-trained ResNet50 model
features_resnet = []
labels = []

for index, row in train_df.iterrows():
    image_path = os.path.join('/content/train_images', row['ID'])
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match ResNet50 input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    features = resnet.predict(image)
    features_resnet.append(features.flatten())
    labels.append(row['Label'])

features_resnet = np.array(features_resnet)
labels = np.array(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features_resnet, labels, test_size=0.2, random_state=42)

# Train a machine learning model (e.g., SVM) using the extracted features
model = SVC()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

# Use the trained model to make predictions on the test images
test_df = pd.read_csv('Test.csv')
features_test = []

for index, row in test_df.iterrows():
    image_path = os.path.join('/content/test_images', row['ID'])
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match ResNet50 input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    features = resnet.predict(image)
    features_test.append(features.flatten())

features_test = np.array(features_test)

# Make predictions
predictions = model.predict(features_test)

# Save the predictions to a CSV file
test_df['Label'] = predictions
test_df.to_csv('Test_Predictions_SVC_1.0.csv', index=False)

1/1 [==============================] - 0s 21ms/step
Validation Accuracy: 0.8235294117647058
1/1 [==============================] - 0s 22ms/step


In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Load the CSV file containing image names and labels for training
train_df = pd.read_csv('Train.csv')

# Load the images and extract features using a pre-trained ResNet50 model
features_resnet = []
labels = []

for index, row in train_df.iterrows():
    image_path = os.path.join('/content/train_images', row['ID'])
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match ResNet50 input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    features = resnet.predict(image)
    features_resnet.append(features.flatten())
    labels.append(row['Label'])

features_resnet = np.array(features_resnet)
labels = np.array(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features_resnet, labels, test_size=0.2, random_state=42)

# Train a machine learning model (e.g., SVM) using the extracted features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Define the parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

# Perform grid search
grid_search = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Train the model with the best parameters
model = SVC(**best_params)
model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = model.predict(X_val_scaled)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)


# Use the trained model to make predictions on the test images
test_df = pd.read_csv('Test.csv')
features_test = []

for index, row in test_df.iterrows():
    image_path = os.path.join('/content/test_images', row['ID'])
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match ResNet50 input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    features = resnet.predict(image)
    features_test.append(features.flatten())

features_test = np.array(features_test)

# Make predictions
predictions = model.predict(features_test)

# # Save the predictions to a CSV file
# test_df['Label'] = predictions
# test_df.to_csv('Test_Predictions_SVC_1.0.csv', index=False)

1/1 [==============================] - 0s 21ms/step
Validation Accuracy: 0.6470588235294118
1/1 [==============================] - 0s 23ms/step


In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

# Load the CSV file containing image names and labels for training
train_df = pd.read_csv('Train.csv')

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,        # Rotate the image by up to 10 degrees
    width_shift_range=0.1,     # Shift the image horizontally by up to 10% of the width
    height_shift_range=0.1,    # Shift the image vertically by up to 10% of the height
    shear_range=0.1,           # Shear the image by up to 10%
    zoom_range=0.1,            # Zoom in by up to 10%
    horizontal_flip=True,      # Flip the image horizontally
    vertical_flip=False,       # Do not flip the image vertically (assuming cells are oriented in a specific direction)
    fill_mode='nearest'        # Fill in newly created pixels with the nearest pixel value
)

# Load the MobileNet model
mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=mobilenet.input, outputs=GlobalAveragePooling2D()(mobilenet.output))

features_mobilenet = []
labels = []

# Generate augmented images and extract features
for index, row in train_df.iterrows():
    image_path = os.path.join('/content/train_images', row['ID'])
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match MobileNet input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    augmented_images = datagen.flow(image)
    for i in range(7):  # Generate 5 augmented images per original image
        augmented_image = augmented_images.next()
        features = model.predict(augmented_image)
        features_mobilenet.append(features.flatten())
        labels.append(row['Label'])

features_mobilenet = np.array(features_mobilenet)
labels = np.array(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features_mobilenet, labels, test_size=0.2, random_state=42)

# Train an SVM model using the extracted features
model_svm = SVC()
model_svm.fit(X_train, y_train)

# Evaluate the model
y_pred = model_svm.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

# Use the trained model to make predictions on the test images
test_df = pd.read_csv('Test.csv')
features_test = []

for index, row in test_df.iterrows():
    image_path = os.path.join('/content/test_images', row['ID'])
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match MobileNet input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    features = model.predict(image)
    features_test.append(features.flatten())

features_test = np.array(features_test)

# Make predictions
predictions = model_svm.predict(features_test)

# Save the predictions to a CSV file
test_df['Label'] = predictions
test_df.to_csv('Test_Predictions_SVM_with_Data_Augmentation.csv', index=False)


1/1 [==============================] - 0s 19ms/step
Validation Accuracy: 0.9304347826086956
1/1 [==============================] - 0s 18ms/step


In [ ]:
0.8257575757575758
4 -> 0.8181818181818182
5 -> 0.8658536585365854
6 -> 0.8383838383838383
7 -> 0.9043478260869565
8 -> 0.8409090909090909
9 -> 0.831081081081081
10 -> 0.823170731707317

# **Random Forest**

In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

# Load the CSV file containing image names and labels for training
train_df = pd.read_csv('Train.csv')

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the MobileNet model
mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=mobilenet.input, outputs=GlobalAveragePooling2D()(mobilenet.output))

features_mobilenet = []
labels = []

# Generate augmented images and extract features
for index, row in train_df.iterrows():
    image_path = os.path.join('/content/train_images', row['ID'])
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match MobileNet input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    augmented_images = datagen.flow(image)
    for i in range(7):  # Generate 5 augmented images per original image
        augmented_image = augmented_images.next()
        features = model.predict(augmented_image)
        features_mobilenet.append(features.flatten())
        labels.append(row['Label'])

features_mobilenet = np.array(features_mobilenet)
labels = np.array(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features_mobilenet, labels, test_size=0.2, random_state=42)

# Train a Random Forest model using the extracted features
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Evaluate the model
y_pred = model_rf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

# Use the trained model to make predictions on the test images
test_df = pd.read_csv('Test.csv')
features_test = []

for index, row in test_df.iterrows():
    image_path = os.path.join('/content/test_images', row['ID'])
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match MobileNet input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    features = model.predict(image)
    features_test.append(features.flatten())

features_test = np.array(features_test)

# Make predictions
predictions = model_rf.predict(features_test)

# Save the predictions to a CSV file
test_df['Label'] = predictions
test_df.to_csv('Test_Predictions_RF_with_Data_Augmentation.csv', index=False)


1/1 [==============================] - 0s 19ms/step
Validation Accuracy: 0.9391304347826087
1/1 [==============================] - 0s 18ms/step


In [ ]:
5 -> 0.9024390243902439
6 -> 0.9191919191919192
7 -> 0.9304347826086956
8 -> 0.8712121212121212

# **Random Forest + DenseNet**

In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

# Load the CSV file containing image names and labels for training
train_df = pd.read_csv('Train.csv')

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the DenseNet model
densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Model(inputs=densenet.input, outputs=GlobalAveragePooling2D()(densenet.output))

features_densenet = []
labels = []

# Generate augmented images and extract features
for index, row in train_df.iterrows():
    image_path = os.path.join('/content/train_images', row['ID'])
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match DenseNet input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    augmented_images = datagen.flow(image)
    for i in range(7):  # Generate 5 augmented images per original image
        augmented_image = augmented_images.next()
        features = model.predict(augmented_image)
        features_densenet.append(features.flatten())
        labels.append(row['Label'])

features_densenet = np.array(features_densenet)
labels = np.array(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features_densenet, labels, test_size=0.2, random_state=42)

# Train a Random Forest model using the extracted features
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Evaluate the model
y_pred = model_rf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

# Use the trained model to make predictions on the test images
test_df = pd.read_csv('Test.csv')
features_test = []

for index, row in test_df.iterrows():
    image_path = os.path.join('/content/test_images', row['ID'])
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match DenseNet input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    features = model.predict(image)
    features_test.append(features.flatten())

features_test = np.array(features_test)

# Make predictions
predictions = model_rf.predict(features_test)

# Save the predictions to a CSV file
test_df['Label'] = predictions
test_df.to_csv('Test_Predictions_RF_with_DenseNet_and_Data_Augmentation.csv', index=False)


1/1 [==============================] - 0s 26ms/step
Validation Accuracy: 0.9739130434782609
1/1 [==============================] - 0s 30ms/step


In [ ]:
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

# Load the CSV file containing image names and labels for training
train_df = pd.read_csv('Train.csv')

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

features_mobilenet = []
labels = []

# Generate augmented images and extract features
for index, row in train_df.iterrows():
    image_path = os.path.join('/content/train_images', row['ID'])
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match MobileNet input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = image.astype('float32') / 255.0  # Normalize pixel values to [0, 1]

    augmented_images = datagen.flow(image)
    for i in range(5):  # Generate 5 augmented images per original image
        augmented_image = augmented_images.next()
        features = model.predict(augmented_image)
        features_mobilenet.append(features.flatten())
        labels.append(row['Label'])

features_mobilenet = np.array(features_mobilenet)
labels = np.array(labels)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(features_mobilenet, labels, test_size=0.2, random_state=42)

# Train a Random Forest model using the extracted features
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)

# Evaluate the model
y_pred = model_rf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

# Use the trained model to make predictions on the test images
test_df = pd.read_csv('Test.csv')
features_test = []

for index, row in test_df.iterrows():
    image_path = os.path.join('/content/test_images', row['ID'])
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize the image to match MobileNet input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = image.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
    features = model.predict(image)
    features_test.append(features.flatten())

features_test = np.array(features_test)

# Make predictions
predictions = model_rf.predict(features_test)

# Save the predictions to a CSV file
test_df['Label'] = predictions
test_df.to_csv('Test_Predictions_RF_with_Data_Augmentation.csv', index=False)


1/1 [==============================] - 0s 21ms/step
Validation Accuracy: 0.47560975609756095
1/1 [==============================] - 0s 20ms/step
